In this notebook we find paths in Cayley graph for LRX(32) using CayleyPy library, beam search and pre-trained neural network to precdict distances.

Neural network was trained in [this notebook](https://www.kaggle.com/code/mrnnnn/lrx-cayleypy-rl-no-pretr-mdqn-rand-permn-gen/output).

The output of this notebook is used [here](https://www.kaggle.com/code/fedimser/lrx-solution-combined) to construct the full submission for the competoition.

In [ ]:
!pip install git+https://github.com/cayleypy/cayleypy@models

In [ ]:
from cayleypy import PermutationGroups, CayleyGraph, Predictor
import pandas as pd 


graph = CayleyGraph(PermutationGroups.lrx(32))
predictor = Predictor.pretrained(graph)
data = pd.read_csv("/kaggle/input/lrx-discover-math-gods-algorithm/test.csv")
ans_csv = ["permutation,solution"]

total_score=0
for _, row in data.iterrows():
    perm_str = row["permutation"]
    start_state = [int(x) for x in perm_str.split(",")]
    n = len(start_state)
    if n != 32:
        continue

    graph.free_memory() 
    result = graph.beam_search(start_state=start_state, predictor=predictor, beam_width=20000, max_iterations=500, return_path=True)
    if result.path_found:
        sol = result.get_path_as_string()
        ans_csv.append(f'"{perm_str}",{sol}')
        score=len(sol.split("."))
        print("Score: ", score)
        total_score+=score
    else:
        print("Path not found!")

with open("/kaggle/working/beamsearch_32.csv", "w") as f:
    f.write("\n".join(ans_csv))

print("Total score:", total_score)